# `swift_too` module

## Swift_PlanQuery example - querying planned *Swift* observations

### API version = 1.2, `swifttools` version = 3.0.7

#### Author: Jamie A. Kennea (Penn State)

The Swift_PlanQuery class allows for querying the database of observations have have been planned for observation by Swift, otherwise known as the "Pre-Planned Science Timeline" (PPST). This allows the user to find out not only what was planned, but what will be observed by Swift in the future.

Two important caveats:

1. For past observations, what should have been down is mostly an academic exercise, as due to TOO uploads and GRBs, Swift's pre-planned observations can be obliterated. If you want to know what Swift *did* observe, do a Swift_ObsQuery

2. Swift's planning is only done days into the future, so if you want to know what is going to happen weeks from now, you're out of luck.

This therefore is mostly useful if you want to know what Swift is observing *today* and *tomorrow*.

In [ ]:
from datetime import datetime, timedelta

from swifttools.swift_too import PlanQuery

### First Query

Note: as of `swifttools 2.2`, anonymous login is the default, so if you do not give the `username` and `shared_secret`, it will log in as 'anonymous'

First query example: What is Swift planned to observe now? Quite easy to construct. Note as of `swifttools` 2.3, you can omit the `Swift_` from class names. Here is our first query:

In [ ]:
plan = PlanQuery(begin=datetime.utcnow())
plan

So the above should have reported back a single target. When you just give `begin` with no `end`, it simply returns the observation which is taking place at the time given. So what is happening at this time tomorrow?

In [ ]:
plan = PlanQuery(begin=datetime.utcnow() + timedelta(days=1))
plan

Did the above query return anything? If not, then there simply isn't a plan for tomorrow. So how can we determine how far into the future Swift has a plan for? The `ppstmax` parameter. Let's check it out:

In [ ]:
f"PPSTs available up to {plan.ppstmax}"

This gives the date of the last planned target. If the query for what Swift is observing this time tomorrow didn't return anything, this date is probably the end of the UT day today. If the time you're running this is before midday on a Monday-Thursday, this is as expected. Typically the plan for tomorrow is submitted for upload around 12pm Eastern time. The exception to this rule is for the weekend plan. Typically on Thursday at 12pm we submit a plan for Friday and Saturday, and on Friday at 12pm we submit a plan for Sunday-Monday. This allows us to cover the weekend, while working a 5 day week.

### Second query example

Seeing a whole day's PPST. So let's see the whole day's Plan that goes farthest into the future.

In [ ]:
dayplan = PlanQuery(begin=plan.ppstmax - timedelta(days=1), end=plan.ppstmax)
dayplan

That's a lot of targets! Note that like Swift_ObsQuery it's showing you all the individual snapshots during the day. If you want to group these into observations, you can do that. Remember the `observations` method returns a dictionary, with obsid as the key, and the total observation details as the value. You can display it as a table in Jupyter Notebooks simply:

In [ ]:
dayplan.observations

### A note about time formats

#### New features in `swifttools` 2.4

All times returned by `PlanQuery` come out in a time system that is derived from Swift's internal clock. The problem with this is that this clock, although close to UTC, is not actually UTC, due to the lack of handling of leap seconds, and also a slow drift in the clock itself. However, you can now correct this using the `clock_correct()` method. Let's do that:

In [ ]:
query = PlanQuery(obsid="03102995005")
query.clock_correct()
query

So the table above now shows times that are specifically labelled as UTC. You'll note that the times now have fractions of a second, which is due to the clock correction that is applied. You can see the value of this clock correction (which includes both leap seconds and clock drift corrections), by looking at one of the date values, e.g.:

In [ ]:
query[0].begin

So for this observation, the UT Correction Factor (UTCF) is -24.012318s. For more detail on this correction, please take a look at the `Swift_Clock` and the notebook explaining that.